In [ ]:
# default_exp core

# module name here

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from torchvision.io.image import read_image
from torchvision.transforms.functional import normalize, resize, to_pil_image
from torchcam.methods import SmoothGradCAMpp
import matplotlib.pyplot as plt
from torchcam.utils import overlay_mask

from sklearn.metrics import confusion_matrix, classification_report
import shutil
import os
from pathlib import Path
from fastai.vision.all import *

In [ ]:
#export
def get_labels_and_preds(predictions):
    df = pd.DataFrame(predictions[0])
    preds = (df[0] < df[1]).astype(int)
    labels = predictions[1]
    return labels, preds

def visualize_cam_on_img(img_name, mymodel):
    model = mymodel
    cam_extractor = SmoothGradCAMpp(model)
    img = read_image(str(img_name))    
    input_tensor = normalize(resize(img, (224, 224)) / 255., [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]).cuda()

    # Preprocess your data and feed it to the model
    out = model(input_tensor.unsqueeze(0))
    # Retrieve the CAM by passing the class index and the model output
    activation_map = cam_extractor(out.squeeze(0).argmax().item(), out)


    # Resize the CAM and overlay it
    result = overlay_mask(to_pil_image(img), to_pil_image(activation_map[0], mode='F'), alpha=0.5)
    # Display it
    plt.imshow(result); plt.axis('off'); plt.tight_layout(); plt.show()


def rename_in_order(folder):
    files = list(folder.glob("*"))
    temp_folder = folder / "temp"
    temp_folder.mkdir()
    target_files = []
    for i in range(len(files)):
        source = files[i]
        target = temp_folder / f"{i+1:03}.jpg"
        target_files.append(target)
        shutil.move(source, target)
    for file in target_files:
        shutil.copy2(file, folder)
    shutil.rmtree(temp_folder)

def init_images_dir(images):
    if images.exists():
        shutil.rmtree(images)

    images.mkdir(parents=True)

    train_folder = images / "train"
    valid_folder = images / "valid"
    test_folder = images / "test"

    train_folder.mkdir()
    valid_folder.mkdir()
    test_folder.mkdir()

def split_images_into_train_valid_test(mycls, images, images_original, train_ratio=0.7, test_ratio=0.1):

    files = list((images_original / mycls).glob("*.jpg"))

    train_length = round(len(files) * train_ratio)
    test_length = round(len(files) * test_ratio)

    train = files[:train_length]
    valid = files[train_length:len(files) - test_length]
    test = files[len(files) - test_length:]

    train_folder = images / "train"
    valid_folder = images / "valid"
    test_folder = images / "test"

    (train_folder / mycls).mkdir()
    (valid_folder / mycls).mkdir()
    (test_folder / mycls).mkdir()

    for file in train:
        shutil.copy(file, train_folder / mycls)

    for file in valid:
        shutil.copy(file, valid_folder / mycls)

    for file in test:
        shutil.copy(file, test_folder / mycls)
        
    rename_in_order(train_folder / mycls)
    rename_in_order(valid_folder / mycls)
    rename_in_order(test_folder / mycls)

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
